## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### План Исследования

#### Поделим наще исследование на несколько частей.

#### Часть 1. Анализ общей информации
<a href='#Анализ общей информации'> 1.1 Загрузка библиотек, чтение данных и анализ общей информации</a>
#### Часть 2. Предобработка данных
<a href='#Обработка выявленных проблем в данных'> 2.1 Обработка выявленных проблем в данных</a>

<a href='#Обработка пропусков'> 2.2 Обработка пропусков</a>

<a href='#Замена типа данных'> 2.3 Замена типа данных</a>

<a href='#Обработка дубликатов'> 2.4 Обработка дубликатов</a>
#### Часть 3. Лемматизация данных
<a href='#Лемматизация'> 3.1 Лемматизация</a>
#### Часть 4. Категоризация данных
<a href='#Категоризация'> 4.1 Категоризация</a>
#### Часть 5. Ответы на целевые вопросы исследования
<a href='#Ответы на вопросы'> 5.1 Ответы на вопросы</a>
#### Часть 6.Общий вывод
<a href='#Общий вывод'> 6.1 Общий вывод</a>

<a id='Анализ общей информации'></a>

### Часть 1. Анализ общей информации. 

In [1]:
# Для анализа данных будем использовать библиотеку pandas_profiling.
# Установим библиотеку и изменим версию pandas c которой profiling работает стабильно
!pip install pandas_profiling==1.4.1
!pip install pandas==0.25.3

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Импортируем библиотеки, с помощью которых будем обрабатывать данные
import pandas_profiling # для анализа статистик и распределений данных
import pandas as pd # для работы с табличными данными
from pymystem3 import Mystem #для обработки категориальных признаков с помощью лемматизации

In [3]:
# Прочитаем данные из файла и запишем в переменную df
df = pd.read_csv('data.csv')

In [4]:
# Выведем на экран первые 10 строк набора данных 
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
# Запусим библиотеку pandas_profiling и посмотрим как выглядят статистики 
pandas_profiling.ProfileReport(df)

/home/jovyan/.local/lib/python3.7/site-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,12
Number of observations,21525
Total Missing (%),1.7%
Total size in memory,2.0 MiB
Average record size in memory,96.0 B
Numeric,6
Categorical,5
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [6]:
# Проверим типы данных у признаков
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [7]:
# Взглянем на строки набора данных, в которых есть пропуски в признаках days_employed и total_income
df_NaN = df[(df['days_employed'].isnull()) & (df['total_income'].isnull())]
df_NaN.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [8]:
# Посмотрим более внимательно на строки, которые задублировались
df[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,покупка жилья для семьи
4182,1,NaN,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,NaN,свадьба
4851,0,NaN,60,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,свадьба
5557,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,сыграть свадьбу
7808,0,NaN,57,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,на проведение свадьбы
8583,0,NaN,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,NaN,дополнительное образование
9238,2,NaN,34,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,покупка жилья для сдачи
9528,0,NaN,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,NaN,операции со своей недвижимостью
9627,0,NaN,56,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,операции со своей недвижимостью
10462,0,NaN,62,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,покупка коммерческой недвижимости


In [9]:
# Посмотрим более подробно на вариации значений в некоторых категориальных признаках
print(df['education'].value_counts())
print(df['purpose'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64
свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной нед

In [10]:
# Т.к. выше мы увидели, что в признаке "dob_years" есть нулевые значения, то следует проверить есть ли еще значения ниже 18
df.loc[(df['dob_years'] > 0) & (df['dob_years'] < 18)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [11]:
# Т.к. мы увидели, что в признаках family_status и family_status_id одинаковое количество категорий, проверим их на соответствие
df.groupby(['family_status_id', 'family_status'])['family_status'].agg(['count'])

,,count
family_status_id,family_status,
0,женат / замужем,12380
1,гражданский брак,4177
2,вдовец / вдова,960
3,в разводе,1195
4,Не женат / не замужем,2813


In [12]:
# Проверим имена колонок на наличие в них пробелов, и в каком регистре они записанны
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

### Вывод

Ознакомившись с набором данных мы видим, что в нём 21525 строк и 12 признаков.

Посмотрим какая информация содержится в каждой колонке:
    
    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита

Из них:
    
    две колонки типа - float64
    пять колонок типа - int64
    пять колонок типа - object

Общие особенности данных:  
Пропущенных значений 1.7% - в колонках 'days_emloyed' и 'total_income' отсутствует одинаковое количество значений - 2174.  
    Одинаковое количество пропущенных значений в этих колонках может свидетельствовать о том, что пропуски не случайны. Возможно заявители специально не указывали свой стаж и размер заработной платы. Других закономерностей не выявлено. Данная категория клиентов находится в группе максимального риска невозврата. В связи с чем данные пропуски будут заменены средними значениями в зависимости от категории.

В данных 54 дубликата строк. Замечено, что во всех задублированных строках отсутствуют данные в колонках 'days_emloyed' и 'total_income'. Предположительно задвоение обусловлено несколькими попытками подачи заявок. 

Анализ признаков:
    
    children — минимальное значение "-1", максимальное значение "20" (скорее всего ошибки ввода данных)
    days_employed — судя по значеням показатель отражает стаж не в днях, а в часах. 2174 неслучайных пропуска. Отрицательные значения (скорее всего ошибки ввода данных). Тип данных float64 (логичнее перевести в int и считать целыми днями).
    dob_years — есть значение "0" (требуется замена нулевых значений на медианное, т.к. клиент банка не может быть младше 18 лет, а удалять строки не желательно). Требуется категоризация.
    education — названия написаны в разных регистрах (ошибки ввода). Требуется привести к нижнему регистру.
    education_id — после приведения значений признака "education" к нижнему регистру проверить их на соответствие и если будут соответствовать, то удалить эту колонку, т.к. в противном случае это дублирование данных
    family_status и family_status_id — соответствуют, а значит колонку с id можно удалить, т.к. это дублирование данных. 
    gender — выявлен один объект со значением "XNA" (ошибка ввода, т.к. среди известных вариаций полов, таких аббревиатур нет.  Перевести в любой из имеющихся, т.к. по другим имеющимся признакам невозможно понять реальное значение этого признака для объекта).
    income_type — имеются малочисленные категории по 1-2 значений (безработный, предприниматель, студент, в декрете), можно объединить в категорию "прочие". В случае критичного изменения данных и появления значительного количества объектов из этих категорий - вывести их. 
    total_income — ип данных float64 (логичнее перевести в int). Требуется категоризация.
    purpose — большое количество вариаций написания одинаковых целей. Требуется категоризация с помощью лемматизации. 




### Часть 2. Предобработка данных

<a id='Обработка выявленных проблем в данных'></a>

### Обработка выявленных проблем в данных

In [13]:
# Заменим значение XNA в признаке gender на наиболее распространенное F
df['gender'] = df['gender'].replace('XNA', 'F')

In [14]:
# Преобразуем значения колонки 'children' из отрицательных в положительные взяв модуль числа
# Заменим выявленные ошибки ввода значений в признаке children на очевидные
df['children'] = df['children'].abs()
df['children'] = df['children'].replace(20, 2)

In [15]:
# Преобразуем значения колонки 'days_employed' из отрицательных в положительные взяв модуль числа
df['days_employed'] = df['days_employed'].abs()

In [16]:
# В признаке days_employed должен храниться стаж в днях, а по факту в часах 
# Переведем часы в дни 
df['days_employed'] = df['days_employed']/24

In [17]:
# Различные вариации названий признака названия education приводим к нижнему регистру
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [18]:
# Похоже, что в признаках education_id и education одинаковое количество категорий, проверим их на соответствие
df.groupby(['education_id', 'education'])['education'].agg(['count'])

,,count
education_id,education,
0,высшее,5260
1,среднее,15233
2,неоконченное высшее,744
3,начальное,282
4,ученая степень,6


In [19]:
# Т.к. выше мы выявили, что у нас есть дублирующие колонки education и family_status, то удалим только колонки с id
df.drop(['education_id', 'family_status_id'], axis=1, inplace=True)
df.head()

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose
0,1,351.569709,42,высшее,женат / замужем,F,сотрудник,0,253875.639453,покупка жилья
1,1,167.700156,36,среднее,женат / замужем,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,234.309275,33,среднее,женат / замужем,M,сотрудник,0,145885.952297,покупка жилья
3,3,171.864467,32,среднее,женат / замужем,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,гражданский брак,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Исправлены ошибки ввода данных в признаках gender, children, days_employed. Удалены дублирующие колонки education и family_status

<a id='Обработка пропусков'></a>

### Обработка пропусков

In [20]:
# Напишем функцию, которая заменяет пропуски на значение среднего по категории, в зависимости от категории строки с пропуском
def replace_nan(column_name, income_type):
    """
        На вход принимает:
        column_name - название колонки, в которой есть пропуски, значение типа str
        income_type - тип категории, на основании которого будем устанавливать фильтр и вычислять значение на замену, значение типа str
        Производится замена значения в исходном наборе данных
        """
    df.loc[(df[column_name].isnull()) & (df['income_type'] == income_type), column_name] = df[df['income_type'] == income_type][column_name].mean()            

In [21]:
# Формируем список уникальных категорий, по строкам в которых есть NaN
# Идем циклом по сформированному списку и вызываем функцию замены пропусков для столбцов 'total_income' и 'days_employed'
# Убедимся, что в данных больше нет пропусков методом info()
list_for_replace_nan = df[df['total_income'].isnull()]['income_type'].unique()
for i in list_for_replace_nan:
    replace_nan('total_income', i)
    replace_nan('days_employed', i)
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 10 columns):
children         21525 non-null int64
days_employed    21525 non-null float64
dob_years        21525 non-null int64
education        21525 non-null object
family_status    21525 non-null object
gender           21525 non-null object
income_type      21525 non-null object
debt             21525 non-null int64
total_income     21525 non-null float64
purpose          21525 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 1.6+ MB


### Вывод

Имевшиеся пропуски значений в колонках 'days_employed' и 'total_income' обработаны.

<a id='Замена типа данных'></a>

### Замена типа данных

In [22]:
# У колонок 'days_employed' и 'total_income' изменим тип с 'float64' на 'int64', 
# т.к. с этими значениями удобнее работать как с целочисленными
# используем метод astype, т.к. он дает возможность указать размерность (64, 32, 16), 
# что может стать критичным для некоторых алгоритмов ML
df['days_employed'] = df['days_employed'].astype('int64')
df['total_income'] = df['total_income'].astype('int64')

### Вывод

Признаки 'days_employed' и 'total_income' нет необходимости хранить в виде значений с плавающей точкой. 

<a id='Обработка дубликатов'></a>

### Обработка дубликатов

In [23]:
# В наборе данных 54 дубликата (выявлено библиотекой pandas_profiler, 
# но можно и методом df.duplicated().sum()) 
# Удалим дубликаты с помощью метода drop_duplicates() 
# Данный метод позволяет сбросить индексы после удаления строк
# Проверим результат
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

### Вывод

Имеющиеся дубликаты обработаны.

### Часть 3. Лемматизация данных

<a id='Лемматизация'></a>

### Лемматизация

In [24]:
# Для каждой строки в наборе данных, в колонке purpose рассчитываем леммы и результат сохраняем в колонке lemmas_purpose
m = Mystem()
df['lemmas_purpose'] = df['purpose'].apply(m.lemmatize)
df.head()

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,lemmas_purpose
0,1,351,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,167,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,234,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,171,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,14177,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


In [25]:
# Пишем функцию для категоризации колонки purpose на 5 распространенных категорий
def get_group(x):
    if 'недвижимость' in x or 'жилье' in x:
        return 'недвижимость'
    if 'образование' in x:
        return 'образование'
    if 'автомобиль' in x:
        return 'автомобиль'
    if 'свадьба' in x:
        return 'свадьба'
    return 'другое'

In [26]:
# Применяем функцию get_group к колонке lemmas_purpose и 
# результат сохраняем в колонку purpose_group
# удаляем временную колонку lemmas_purpose
df['purpose_group'] = df['lemmas_purpose'].apply(get_group)
df.drop(['lemmas_purpose'], axis=1, inplace=True)
df.head()

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,purpose_group
0,1,351,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,167,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,234,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,171,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### Вывод

В признаке purpose у нас хранилось очень много вариаций написания одинаковых целей. 
Для исправления этой ситуации требуется категоризация. 
В данной задаче удобнее всего использовать автоматическую библиотеку pymystem3, которая обрабатывает строки, создавая списки лемм. 
Из получившихся списков лемм выбираем ключевые слова для отнесения к той или иной категории по условию вхождения в список. 

### Часть 4. Категоризация данных

<a id='Категоризация'></a>

### Категоризация данных

In [27]:
# Напишем функцию для категоризации возраста по 4 группам
# Группы разделены по принципу студенты, средний возраст, взрослые, пенсионеры
def years_group(age):
        """
        На вход принимает возраст, значение типа int
        Возвращает принадлежность к одной из возрастных групп, используя правила:
        - '18-25' при значении < 25 лет, включая 25
        - '26-45' при значениии более 25 и менее 45, включая 45
        - '46-65' при значениии более 45 и менее 65, включая 65
        - '65 и старше' во всех остальных случаях.
        """

        if age <= 25:
                return '18-25'
        if age <= 45:
                return '26-45'
        if age <= 65:
                return '46-65'
        return '65 и выше'

In [28]:
# Применяем функцию years_group к данным колонки dob_years. Результат сохраняем в колонке dob_years_group
df['dob_years_group'] = df['dob_years'].apply(years_group)
df.head()

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,purpose_group,dob_years_group
0,1,351,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,недвижимость,26-45
1,1,167,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,26-45
2,0,234,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,недвижимость,26-45
3,3,171,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,образование,26-45
4,0,14177,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,свадьба,46-65


In [29]:
# Напишем функцию для категоризации количества детей на 3 группы
# Группы разделены по принципу без детей, среднее количество детей, многодетные
def children_group(children):
        """
        На вход принимает количество детей, значение типа int
        Возвращает принадлежность к одной из групп, используя правила:
        - '0' при значении равном 0 детей
        - '1-3' при значениии более 1 но не более 3, включая 3
        - 'больше 3' во всех остальных случаях
        """

        if children == 0:
                return '0'
        if children <= 3:
                return '1-3'
        return 'больше 3'

In [30]:
# Применяем функцию children_group к данным колонки children. Результат сохраняем в колонке children_group
df['children_group'] = df['children'].apply(children_group)
df.head()

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,purpose_group,dob_years_group,children_group
0,1,351,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,недвижимость,26-45,1-3
1,1,167,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,26-45,1-3
2,0,234,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,недвижимость,26-45,0
3,3,171,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,образование,26-45,1-3
4,0,14177,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,свадьба,46-65,0


In [31]:
# Напишем функцию для категоризации уровня зарплат на 5 групп
# Группы разделены по принципу низкий доход, ниже среднего, средний, выше среднего, высокий
def total_income_group(total_income):
        """
        На вход принимает уровень зарплаты, значение типа int
        Возвращает принадлежность к одной из групп, используя правила:
        - '0-75000' при значении total_income <= 75000
        - '76000-100000' при значениии total_income более 76000 но не более 100000, включая 100000
        - '101000-200000' при значениии total_income более 101000 но не болеее 200000, включая 200000
        - '201000-300000' при значениии total_income более 201000 но не более 300000, включая 300000
        - '301000 и более' при значениии total_income более 301000
        """

        if total_income <= 75000:
                return '0-75000'
        if total_income <= 100000:
                return '76000-100000'
        if total_income <= 200000:
                return '101000-200000'
        if total_income <= 300000:
                return '201000-300000'
        return '301000 и более'

In [32]:
# Применяем функцию total_income_group к данным колонки total_income. Результат сохраняем в колонке total_income_group
df['total_income_group'] = df['total_income'].apply(total_income_group)
df.head()

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,purpose_group,dob_years_group,children_group,total_income_group
0,1,351,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,недвижимость,26-45,1-3,201000-300000
1,1,167,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,26-45,1-3,101000-200000
2,0,234,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,недвижимость,26-45,0,101000-200000
3,3,171,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,образование,26-45,1-3,201000-300000
4,0,14177,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,свадьба,46-65,0,101000-200000


In [33]:
# Объединим малочисленные значения признаков колонки income_type ('безработный','предприниматель','студент','в декрете') 
# в категорию "прочие"
df['income_type'] = df['income_type'].replace(['безработный','предприниматель','студент','в декрете'], 'прочие')
df['income_type'].value_counts()

сотрудник      11084
компаньон       5078
пенсионер       3829
госслужащий     1457
прочие             6
Name: income_type, dtype: int64

### Вывод

Произведена категоризация признаков  dob_years, children, total_income, income_type

### Часть 5. Ответы на целевые вопросы исследования

<a id='Ответы на вопросы'></a>

### - Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [34]:
family_pivot = df.pivot_table(index=['children_group'], columns='debt', values='purpose', aggfunc='count')
family_pivot['debt_count'] = family_pivot[0] + family_pivot[1] 
family_pivot['ratio'] = family_pivot[1]/(family_pivot[0] + family_pivot[1]) * 100
family_pivot.columns = ['debt_0', 'debt_1','debt_count','ratio']
print(family_pivot[['debt_count', 'debt_1' ,'ratio']], '\n') 

print('{:.1f}% бездетных заёмщиков имели задолженность по кредиту'.format(family_pivot['ratio'][0]))
print('{:.1f}% заёмщиков со средним количеством детей имели задолженность по кредиту'.format(family_pivot['ratio'][1]))
print('{:.1f}% многодетных заёмщиков имели задолженность по кредиту'.format(family_pivot['ratio'][2]))

                debt_count  debt_1     ratio
children_group                              
0                    14091    1063  7.543822
1-3                   7313     674  9.216464
больше 3                50       4  8.000000 

7.5% бездетных заёмщиков имели задолженность по кредиту
9.2% заёмщиков со средним количеством детей имели задолженность по кредиту
8.0% многодетных заёмщиков имели задолженность по кредиту


### Вывод

Зависимость есть - наличие детей от 1 до 3 увеличивает риск невозврата кредита

### - Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
family_pivot = df.pivot_table(index=['family_status'], columns='debt', values='purpose', aggfunc='count')
family_pivot['debt_count'] = family_pivot[0] + family_pivot[1] 
family_pivot['ratio'] = family_pivot[1]/(family_pivot[0] + family_pivot[1]) * 100
family_pivot.columns = ['debt_0', 'debt_1','debt_count','ratio']
print(family_pivot[['debt_count', 'debt_1' ,'ratio']], '\n') 

print('{:.1f}% неженатых / незамужних заёмщиков имели задолженность по кредиту'.format(family_pivot['ratio'][0]))
print('{:.1f}% заёмщиков в разводе имели задолженность по кредиту'.format(family_pivot['ratio'][1]))
print('{:.1f}% заёмщиков среди вдовцов / вдов имели задолженность по кредиту'.format(family_pivot['ratio'][2]))
print('{:.1f}% заёмщиков в гражданском браке имели задолженность по кредиту'.format(family_pivot['ratio'][3]))
print('{:.1f}% женатых / замужних заёмщиков имели задолженность по кредиту'.format(family_pivot['ratio'][4]))

                       debt_count  debt_1     ratio
family_status                                      
Не женат / не замужем        2810     274  9.750890
в разводе                    1195      85  7.112971
вдовец / вдова                959      63  6.569343
гражданский брак             4151     388  9.347145
женат / замужем             12339     931  7.545182 

9.8% неженатых / незамужних заёмщиков имели задолженность по кредиту
7.1% заёмщиков в разводе имели задолженность по кредиту
6.6% заёмщиков среди вдовцов / вдов имели задолженность по кредиту
9.3% заёмщиков в гражданском браке имели задолженность по кредиту
7.5% женатых / замужних заёмщиков имели задолженность по кредиту


### Вывод

Зависимость есть - выделяются категории семейного положения "Не женат / не замужем" и "гражданский брак", которые чаще, по сравнению с другими, не возвращают кредит.

### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
family_pivot = df.pivot_table(index=['total_income_group'], columns='debt', values='purpose', aggfunc='count')
family_pivot['debt_count'] = family_pivot[0] + family_pivot[1] 
family_pivot['ratio'] = family_pivot[1]/(family_pivot[0] + family_pivot[1]) * 100
family_pivot.columns = ['debt_0', 'debt_1','debt_count','ratio']
print(family_pivot[['debt_count', 'debt_1' ,'ratio']], '\n') 

print('{:.1f}% заёмщиков с заработком до 75 000 имели задолженность по кредиту'.format(family_pivot['ratio'][0]))
print('{:.1f}% заёмщиков с заработком в диапазоне 76 000-100 000  имели задолженность по кредиту'.format(family_pivot['ratio'][4]))
print('{:.1f}% заёмщиков с заработком в диапазоне 101 000-200 000 имели задолженность по кредиту'.format(family_pivot['ratio'][1]))
print('{:.1f}% заёмщиков с заработком в диапазоне 201 000-300 000 имели задолженность по кредиту'.format(family_pivot['ratio'][2]))
print('{:.1f}% заёмщиков с заработком более 301 000 имели задолженность по кредиту'.format(family_pivot['ratio'][3]))

                    debt_count  debt_1     ratio
total_income_group                              
0-75000                   1865     136  7.292225
101000-200000            11423     999  8.745513
201000-300000             4085     282  6.903305
301000 и более            1483     106  7.147674
76000-100000              2598     218  8.391070 

7.3% заёмщиков с заработком до 75 000 имели задолженность по кредиту
8.4% заёмщиков с заработком в диапазоне 76 000-100 000  имели задолженность по кредиту
8.7% заёмщиков с заработком в диапазоне 101 000-200 000 имели задолженность по кредиту
6.9% заёмщиков с заработком в диапазоне 201 000-300 000 имели задолженность по кредиту
7.1% заёмщиков с заработком более 301 000 имели задолженность по кредиту


### Вывод

Зависимость есть - заёмщики с уровнем зарплаты в диапазоне 76 000 - 200 000 чаще не возвращают кредит в срок.

### - Как разные цели кредита влияют на его возврат в срок?

In [37]:
family_pivot = df.pivot_table(index=['purpose_group'], columns='debt', values='purpose', aggfunc='count')
family_pivot['debt_count'] = family_pivot[0] + family_pivot[1] 
family_pivot['ratio'] = family_pivot[1]/(family_pivot[0] + family_pivot[1]) * 100
family_pivot.columns = ['debt_0', 'debt_1','debt_count','ratio']
print(family_pivot[['debt_count', 'debt_1' ,'ratio']], '\n') 

print('{:.1f}% заёмщиков с целью кредита на приобретение автомобиля имели задолженность по кредиту'.format(family_pivot['ratio'][0]))
print('{:.1f}% заёмщиков с целью кредита на приобретение недвижимости имели задолженность по кредиту'.format(family_pivot['ratio'][1]))
print('{:.1f}% заёмщиков с целью кредита на получение образования имели задолженность по кредиту'.format(family_pivot['ratio'][2]))
print('{:.1f}% заёмщиков с целью кредита на организацию свадьбы имели задолженность по кредиту'.format(family_pivot['ratio'][3]))

               debt_count  debt_1     ratio
purpose_group                              
автомобиль           4306     403  9.359034
недвижимость        10811     782  7.233373
образование          4013     370  9.220035
свадьба              2324     186  8.003442 

9.4% заёмщиков с целью кредита на приобретение автомобиля имели задолженность по кредиту
7.2% заёмщиков с целью кредита на приобретение недвижимости имели задолженность по кредиту
9.2% заёмщиков с целью кредита на получение образования имели задолженность по кредиту
8.0% заёмщиков с целью кредита на организацию свадьбы имели задолженность по кредиту


### Вывод

Цели кредита на приобретение автомобиля и получение образования увеливает риск невозврата кредита в срок.

<a id='Общий вывод'></a>

### Часть 6. Общий вывод

Анализ данных показал наличие ошибок ввода данных, сознательное неуказание значений в колонках "количество отработанных дней" и "уровень заработной платы". Профиль клиента невернувшего кредит выглядит как - вне официального брака, с детьми в количестве от 1 до 3, с уровнем заработной платы в диапазоне 76000-200000, взявший кредит на покупку автомобиля или получение образования. 